In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown **WebUI Version**
TagVersion = 'forge' #@param ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0", "v1.7.0", "v1.8.0", "forge"]

#@markdown -----
#@markdown *터널링*

#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

#@markdown **Localtunnel**
Localtunnel = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/lycoris : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = '#template_checkpoint_default#' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = '#template_checkpoint_default_name#' #@param {type:"string"}

#@markdown -----
#@markdown *추가익스텐션*

#@markdown **ControlNet을 사용하지 않으면 체크를 해제해주세요**
#@markdown > *openpose, lineart, softedge, depth, canny, tile, ip2p*  
ControlNet = True #@param {type:"boolean"}

#@markdown > *temporalnet, segment, inpaint, scribble, mlsd, normalbae, suffle*  
ControlNet_Extra = True #@param {type:"boolean"}

#@markdown **IP-Adapter 모델 다운로드**
CotnrolNet_IPAdapter = False #@param {type:"boolean"}

#@markdown **T2I Adapter 모델 다운로드**
#@markdown > *style, color, sketch, zoedepth, recolor*  
ControlNet_T2I = False #@param {type:"boolean"}

#@markdown **After Detailer(adetailer)를 사용하지 않으면 체크를 해제해주세요**
ADetailer = True #@param {type:"boolean"}

#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = False #@param {type:"boolean"}

#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False #@param {type:"boolean"}

#@markdown **AnimateDiff 사용 (WebUI 버전 v1.6.0만 지원)**
AnimateDiff = False #@param {type:"boolean"}

#@markdown **AnimateDiff 추가 모션 모델 (HumansMotion,3DMotion)**
AnimateDiff_Extra = False #@param {type:"boolean"}

#@markdown **mov2mov 사용**
Mov2mov = False #@param {type:"boolean"}

#@markdown **Deforum 사용**
Deforum = False #@param {type:"boolean"}

#@markdown **Reactor 사용**
Reactor = False #@param {type:"boolean"}

Workspace = 'ui'
NotebookVersion = '#template_notebook#'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion/google'

!apt -y install -qq aria2

%cd /content

# 설치 
import base64
webui = base64.b64decode(b'c2Qtd2VidWk=').decode('utf-8')

# 필요한 패키지 설치
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_torch.sh -O /content/install.sh
!bash /content/install.sh

!pip install httpx==0.24.1

# DDetailer가 True이면 openmim, mmcv, mmdet 패키지 설치
if DDetailer:
    !pip install -U openmim
    !mim install mmcv>=2.0.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
    !mim install mmdet>=3.0.0

# TagVersion이 forge이면 forgeui를, 아니면 해당 버전의 colabui를 클론
if TagVersion == "forge":
    !git clone https://github.com/ninjaneural/forgeui /content/{Workspace}
else:
    !git clone -b {TagVersion} https://github.com/ninjaneural/colabui /content/{Workspace}

# 작업 디렉토리 이동 및 불필요한 파일 삭제
%cd /content/{Workspace}
!rm webui*.sh

# 최신 버전 여부 확인
latest_version = TagVersion in ["v1.6.0", "v1.7.0", "v1.8.0", "forge"]

if ControlNet and TagVersion != "forge":
    !git clone https://github.com/Mikubill/{webui}-controlnet ./extensions/controlnet

if DDetailer:
    !git clone -b mmdet3 https://github.com/ninjaneural/ddetailer ./extensions/ddetailer

if ADetailer:
    !git clone https://github.com/Bing-su/adetailer ./extensions/adetailer

if SegmentAnything:
    !git clone https://github.com/ninjaneural/segment-anything ./extensions/segment-anything

# 나머지 확장 프로그램들
!git clone https://github.com/adieyal/sd-dynamic-prompts ./extensions/sd-dynamic-prompts
!git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding ./extensions/sd-dynamic-thresholding
if TagVersion != "forge":
  !git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 ./extensions/multidiffusion-upscaler

!git clone https://github.com/DominikDoom/a1111-{webui}-tagcomplete ./extensions/tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 ./extensions/ultimate-upscale

!git clone https://github.com/neuralninja22/images-browser ./extensions/images-browser
!git clone https://github.com/kohya-ss/{webui}-additional-networks ./extensions/additional-networks
!git clone https://github.com/picobyte/stable-diffusion-webui-wd14-tagger ./extensions/wd14-tagger
!git clone https://github.com/yankooliveira/{webui}-photopea-embed ./extensions/photopea-embed
!git clone https://github.com/ninjaneural/video-util ./extensions/video-util

!git clone https://github.com/fkunn1326/openpose-editor ./extensions/openpose-editor
!git clone https://github.com/hnmr293/posex ./extensions/posex
!git clone https://github.com/fishslot/video_loopback_for_webui ./extensions/video_loopback

if not latest_version:
  !git clone https://github.com/KohakuBlueleaf/a1111-{webui}-lycoris ./extensions/lycoris

if Reactor:
    !git clone https://github.com/Gourieff/{webui}-reactor ./extensions/{webui}-reactor
    !pip install insightface==0.7.3
    !pip install onnxruntime-gpu

if Mov2mov:
    !git clone https://github.com/Scholar01/{webui}-mov2mov ./extensions/mov2mov
    !sed -i -e "/if platform.system() == 'Windows':/if platform.system() == 'Windows' or platform.system() == 'Linux':" ./extensions/mov2mov/script/m2m_util.py

if AnimateDiff:
    !git clone https://github.com/continue-revolution/{webui}-animatediff ./extensions/animatediff
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -d ./extensions/animatediff/model -o mm_sd_v14.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -d ./extensions/animatediff/model -o mm_sd_v15.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d ./extensions/animatediff/model -o mm_sd_v15_v2.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.ckpt -d ./extensions/animatediff/model -o temporaldiff-v1-animatediff.ckpt

if Deforum:
    !git clone -b nightly https://github.com/ninjaneural/deforum ./extensions/deforum

if SegmentAnything:
    !mkdir ./models/sam
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -d ./models/sam -o sam_vit_h_

# 최신 버전이 아니면 추가 작업 실행
if not latest_version:
    !sed -i -e "/from modules import launch_utils/a\import os" /content/{Workspace}/launch.py
    !sed -i -e "/        prepare_environment()/a\        os.system(f\\\"\"\"sed -i -e \"s/dict()))/dict())).cuda()/g\" /content/{Workspace}/repositories/stable-diffusion-stability-ai/ldm/util.py\"\"\")" /content/{Workspace}/launch.py
    !cd ./extensions/controlnet && git reset --hard 3011ff6e706d3fdd0cc7d2ac8ff0d59020b8f767
    !pip install gradio-client==0.8.1

# config.json 다운로드
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/config.json -O ./config.json

# 구글드라이브 연결
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/link_google_drive.sh -O /content/link_google_drive.sh
!bash /content/link_google_drive.sh {Workspace} {Google_Drive} {Google_Drive_Dir} {NotebookVersion}

# 커스터마이징
%cd /content/{Workspace}
!bash /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/install.sh {Workspace}
if Google_Drive:
  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/config.json /content/{Workspace}/config.json
  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/ui-config.json /content/{Workspace}/ui-config.json
  !cp -f /content/drive/MyDrive/{Google_Drive_Dir}/{NotebookVersion}/styles.csv /content/{Workspace}/styles.csv
%cd /content/{Workspace}

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' /content/{Workspace}/config.json

print('download...')
with capture.capture_output() as cap:

  # upscaler
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d ./models/ESRGAN -o 4x-UltraSharp.pth

  # checkpoint
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

  # vae
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d ./models/VAE -o kl-f8-anime2.ckpt

  # lora
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16" -d ./models/Lora -o blindbox_V1Mix.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors" -d ./models/Lora -o lcm-lora-sd15.safetensors

  # embeddings
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d ./embeddings -o EasyNegative.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d ./embeddings -o badhandv4.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d ./embeddings -o bad-hands-5.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d ./embeddings/ -o negative_hand.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d ./embeddings/ -o bad_prompt_version2.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/5637?type=Model&format=PickleTensor&size=full&fp=fp16" -d ./embeddings/ -o ng_deepnegative_v1_75t.pt

  if ControlNet:
    ControlNet_Model_Path = "./extensions/controlnet/models"

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors -d {ControlNet_Model_Path} -o control_sd15_inpaint_depth_hand_fp16.safetensors

    if ControlNet_Extra:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_seg_fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_inpaint_fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_scribble_fp16.safetensors

      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_mlsd_fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_normalbae_fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_shuffle_fp16.safetensors
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {ControlNet_Model_Path} -o diff_control_sd15_temporalnet_fp16.safetensors

    if CotnrolNet_IPAdapter:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter_sd15.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus_sd15.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus-face_sd15.pth

    if ControlNet_T2I:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_style_sd14v1.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_color_sd14v1.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {ControlNet_Model_Path} -o t2iadapter_sketch_sd15v2.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d .{ControlNet_Model_Path} -o t2iadapter_zoedepth_sd15v1.pth
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {ControlNet_Model_Path} -o ioclab_sd15_recolor.safetensors  

  if AnimateDiff_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/174464 -d ./extensions/animatediff/model -o improvedHumansMotion_refinedHumanMovement.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/178017 -d ./extensions/animatediff/model -o improved3DMotion_improved3DV1.ckpt

if Localtunnel:
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

if Ngrok_Key:
  !python launch.py --xformers --no-half-vae --theme dark --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue
